### Import models


In [37]:
from MODELS.ai_0014_MARK1 import SoundClassifier_MARK1
from MODELS.ai_0014_MARK2 import SoundClassifier_MARK2
from MODELS.ai_0014_MARK3 import SoundClassifier_MARK3
from GPU_torch import GPU

In [38]:
GPU()

TypeError: GPU() missing 1 required positional argument: 'void'

In [12]:
import torch
'''SEED Everything'''
import random
import numpy as np
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True # keep True if all the input have same size.
SEED=42
seed_everything(SEED=SEED)

In [13]:
# chaning the last line to 10 number since there is total 10 class
# Define some parameters
num_classes = 10  # for UrbanSound8K dataset, there are 10 classes
input_size = 44100 * 4  # based on the fixed length of the audio samples

### MARK 1

In [14]:

import librosa
import torch
import torch.nn as nn
import sounddevice as sd
import numpy as np
from torchvision.transforms import functional as F
from scipy.io.wavfile import write

# Define the path to the saved model checkpoint
saved_model_path = 'best_model.pth'

# Create the model
model = SoundClassifier_MARK1(input_size, num_classes)

# Load the saved model checkpoint
model.load_state_dict(torch.load(saved_model_path))
model.eval()

# Set the duration and sample rate of recordings
duration = 5  # seconds
sample_rate = 44100  # in hertz, the standard sample rate for UrbanSound8K

def audio_callback(indata, frames, time, status):
    volume_norm = np.linalg.norm(indata) * 10
    print ("\r|" + "#" * int(volume_norm) + "-" * (10 - int(volume_norm)) + "|", end='')

# Create a stream object (sound device)
stream = sd.InputStream(callback=audio_callback, channels=1, samplerate=sample_rate)

# Define the class names
class_names = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']

print("Starting stream...")

with stream:
    while True:
        print("\nListening for audio...")
        # Record audio for a set duration
        recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
        sd.wait()  # Wait for the recording to finish
        print("Recording finished. Preprocessing and classifying...")
        
        # Preprocess the recorded audio (depending on how you've implemented UrbanSoundDataset)
        #processed_recording = UrbanSoundDataset.preprocess(recording)

        # Preprocess the recorded audio
        def preprocess_live_audio(audio):
            # Convert the audio to a spectrogram
            spectrogram = librosa.feature.melspectrogram(audio, sr=sample_rate)

            # Apply the logarithm to the spectrogram
            log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)

            # Your model might expect a specific shape or other properties, so adjust as needed
            # This is a simple example and may not perfectly match your needs

            return torch.Tensor(log_spectrogram)

        # Replace the preprocessing line in the previous code with the new function
        processed_recording = preprocess_live_audio(recording)

        # Run the model on the processed audio
        with torch.no_grad():
            model.eval()
            processed_recording = processed_recording.to(device)
            output = model(processed_recording)

        # Determine the predicted class
        _, predicted_class = torch.max(output.data, 0)
        print(f"Predicted sound class: {class_names[predicted_class.item()]}")
        time.sleep(0.1)  # You can adjust this delay as needed


Starting stream...

Listening for audio...
|----------|

KeyboardInterrupt: 